load datasets from kaggle 

https://www.analyticsvidhya.com/blog/2021/06/how-to-load-kaggle-datasets-directly-into-google-colab/

before first line you need to import kaggle.json into filesystem



In [ ]:
# google colab
#! mkdir ~/.kaggle
#! cp kaggle.json ~/.kaggle/
#! chmod 600 ~/.kaggle/kaggle.json
#! kaggle competitions download cifar-10
#!unzip cifar-10.zip

In [ ]:
#kaggle notebook
!pip install torchsummary
!pip install --upgrade torch torchvision

In [ ]:
!pip install py7zr

In [ ]:
#!python -m py7zr x ./train.7z # colab 
!python -m py7zr x ../input/cifar-10/train.7z #kaggle notebook

In [ ]:
#!python -m py7zr x ./test.7z
!python -m py7zr x ../input/cifar-10/test.7z #kaggle notebook

In [ ]:
#imports
import math
import random
import copy
import os

import pandas as pd
import numpy as np

import torch
from torchvision.io import read_image
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torchsummary import summary
from torchvision import models

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix


import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

from tqdm import tqdm


In [ ]:
#variable
IMG_SIZE = 32

In [ ]:
hyp = {
    #regularization hyperparameters
    "dropout_rate" : 0.3,
    "data_augmentation_rate" : 0.09, # probabilty of any transformation = 1 - (1 - data_augmentation_rate)**(Number of transformation)
    "data_augmentation_rate_mix" : 0.05,



    #leraning hyperparameters
    "batch_size" : 64,
    "lerning_rate" :  1e-3,
    "shuffle" : True
}


In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self,  img_dir,annotations_file = None,transform = None):
      self.img_labels = None
      self.le = preprocessing.LabelEncoder()
      if  annotations_file is not None:
        self.img_labels = pd.read_csv(annotations_file)
        self.img_labels['index_label'] = self.le.fit_transform(self.img_labels.iloc[:,1])
      self.img_dir = img_dir
      self.transform = transform

    def __len__(self):
      if self.img_labels is None:
        return len(os.listdir(self.img_dir))
      return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, str(idx + 1) + '.png')
        image = read_image(img_path)
        label = F.one_hot(torch.tensor(1),10).float()
        if  self.img_labels is not None:
          label = self.img_labels.iloc[idx, 2]
          label = torch.tensor(int(label))
          label = F.one_hot(label,10).float()

        # apply transformations
        if self.transform:
          for t in self.transform:
            image = t(image)
        return image, label
          
    def getLabel(self,idx):
      if  self.img_labels is not None:
        return self.img_labels.iloc[idx,1]
      return None

    def encodeLabel(self, idx):
      if self.img_labels is None:
        return None
      return self.le.inverse_transform(idx)

In [ ]:
class RandomProbabilityWrapper:

    def __init__(self,probability,transformation):
        self.transformation = transformation
        self.probability = probability


    def __call__(self, img):
        if random.random() < self.probability:
            return self.transformation(img)
        return img

In [ ]:
class Cutmix:
  def __init__(self,probability, dataset):
    self.probability = probability
    self.dataset = dataset
    
  def transform(self, batch):
    indexes = np.where(np.random.rand(len(batch[0])) < self.probability)[0]
    print(indexes)
    for i in indexes:
        rand_index = random.randint(0,len(self.dataset) - 1 )
        second_image, second_label = self.dataset[rand_index]
        batch[0][i], batch[1][i] = self(batch[0][i], batch[1][i],second_image, second_label)
    return batch
    


  def __call__(self, image1, label1, image2, label2, beta_coff = 0.3,debug = False):
    ratio = np.random.beta(beta_coff, beta_coff, size=None)
    a = int(math.sqrt(ratio) * IMG_SIZE)
    x = random.randint(0, IMG_SIZE -1)
    y = random.randint(0, IMG_SIZE -1)
    x2 = x + a if x + a < IMG_SIZE else IMG_SIZE
    y2 = y + a if y + a < IMG_SIZE else IMG_SIZE
    width = x2 - x
    height = y2 - y
    ratio = width * height * 1.0 / (IMG_SIZE ** 2)

    crop2 = T.functional.crop(image2, y, x, height, width) #take only selected part 
    image2 = T.functional.pad(crop2, [x, y, IMG_SIZE - x2, IMG_SIZE - y2]) #set it on image dimenzions black image

    crop1 = T.functional.crop(image1, y, x, height, width)
    img1 = T.functional.pad(crop1, [x, y, IMG_SIZE - x2, IMG_SIZE - y2])

    image1 = image1 - img1
    image = image1 + image2

    label = ratio * label2 + (1 - ratio) * label1
    if debug == True:
        print("a = " + str(a))
        print("x, y = " +str(x) + ", "+str(y))
        print("x2, y2 = " +str(x2) + ", "+str(y2))
        print("width, height = " +str(width) + ", "+str(height))
        
    return image, label

In [ ]:
class DataAugmentationLayer(nn.Module):
    def __init__(self, image_transformation = []):
        super().__init__()
        self.image_transformation = image_transformation

    def forward(self, x):
        if self.training == True:
            for t in self.image_transformation:
                x = t(x)          
        return x    

In [ ]:
class TransformationLayer(nn.Module):
    def __init__(self, image_transformation = []):
        super().__init__()
        self.image_transformation = image_transformation

    def forward(self, x):
        for t in self.image_transformation:
            x = t(x)          
        return x    

In [ ]:
#cifar10_dataset = CustomImageDataset("./train/","./trainLabels.csv") # google colab
cifar10_dataset = CustomImageDataset("./train/","../input/cifar-10/trainLabels.csv") #kaggle notebook

In [ ]:
train_datasets, val_datasets = train_test_split(cifar10_dataset, test_size=0.2)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
figure(figsize=(8, 6), dpi=80)
imgNum = 2
arr_ = cifar10_dataset[imgNum][0]
plt.imshow(arr_.T)
plt.show()
print(cifar10_dataset[imgNum][1])
print(cifar10_dataset.getLabel(imgNum))


In [ ]:
rotation = RandomProbabilityWrapper(hyp["data_augmentation_rate"], lambda x : T.functional.rotate(x,int(np.random.choice([90,180,270]))))
resize_cropper = RandomProbabilityWrapper(hyp["data_augmentation_rate"], lambda x :T.RandomResizedCrop(size=(32, 32))(x))
affine = RandomProbabilityWrapper(hyp["data_augmentation_rate"], lambda x :T.RandomAffine(degrees=0, translate=(0.1, 0.4), scale=(1,1))(x))

cutmix = Cutmix(hyp["data_augmentation_rate_mix"],cifar10_dataset)

    

In [ ]:
dataAugumentation = DataAugmentationLayer(image_transformation = [rotation,resize_cropper,affine])

In [ ]:
train_dataloader = DataLoader(train_datasets, batch_size=hyp["batch_size"], shuffle=hyp["shuffle"])
val_dataloader = DataLoader(val_datasets, batch_size=hyp["batch_size"], shuffle=hyp["shuffle"])

In [ ]:
imgNum = random.randint(0,len(cifar10_dataset))
imgNum1 =random.randint(0,len(cifar10_dataset))
figsize=(8, 6)
fig, (ax1, ax2,ax3) = plt.subplots(1, 3,figsize=(35, 10) )
i1,l1 = cifar10_dataset[imgNum]
ax1.imshow(i1.int().T)
ax1.set_title(str(l1) + "--> " + cifar10_dataset.encodeLabel([torch.argmax(l1)]))

i2,l2 = cifar10_dataset[imgNum1]
ax2.imshow(i2.int().T)
ax2.set_title(str(l2) + "--> " + cifar10_dataset.encodeLabel([torch.argmax(l2)]))


cut_test = Cutmix(1,cifar10_dataset)

figure(figsize=(8, 6), dpi=80)
image, label = cut_test(i1,l1,i2,l2,debug = True)
ax3.imshow(image.int().T)
ax3.set_title(str(label) + "--> " + cifar10_dataset.encodeLabel([torch.argmax(label)]))



In [ ]:
def train(model, train_dataloader, test_dataloader = None, num_epochs=1, lr=1e-3, keepBest = False, mix_trainsformation = None):
  optimizer = torch.optim.Adam(model.parameters(), lr=lr)
  array_loss_train = np.empty([0])
  array_loss_val = np.empty([0])
  #with wandb.init(project="UZOP_LAB"):
  #model.cuda(0)
  best_model_wts = None
  best_lost = None

  for epoch in range(num_epochs):
    correct_train = 0
    model.train()
    with tqdm(enumerate(train_dataloader), total=len(train_dataloader), desc=f'Training (epoch={epoch}/{num_epochs})') as epoch_progress:
      for batch_idx, train_batch in epoch_progress:
        if mix_trainsformation is not None:
            for t in mix_trainsformation:
                train_batch = t.transform(train_batch)
                
        x, y = train_batch      
        x = x.float().cuda()
        y = y.cuda()
        logits = model(x)
        value, predicted = torch.max(logits, 1)
        loss = F.cross_entropy(logits, y)
        epoch_correct = (predicted==torch.argmax(y, dim=1)).sum().item()
        correct_train += epoch_correct
        epoch_progress.set_postfix({'loss': 1 - epoch_correct / hyp["batch_size"]})  

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_loss = 1 - (correct_train / len(train_dataloader.dataset))
    array_loss_train = np.append(array_loss_train,train_loss)
    print("Train loss " + str(train_loss))

    if test_dataloader is not None: 
      with torch.no_grad():
        model.eval()
        correct_val = 0
        for test_batch in tqdm(test_dataloader, desc="Testing"):
          x, y = test_batch
          x = x.float().cuda()
          y = y.cuda()

          logits = model(x)
          value, predicted = torch.max(logits, 1)
          loss = F.cross_entropy(logits, y)
          epoch_correct = (predicted==torch.argmax(y, dim=1)).sum().item()
          correct_val += epoch_correct

        val_loss = 1 - (correct_val / len(test_dataloader.dataset))
        array_loss_val = np.append(array_loss_val,val_loss)
        print("Val loss " + str(val_loss))

      if best_lost is None or best_lost > val_loss:
            best_lost = val_loss
            best_model_wts = copy.deepcopy(model.state_dict())

  if test_dataloader is not None: 
    if keepBest:
        model.load_state_dict(best_model_wts)
      
    model.eval()
    
    with torch.no_grad():
      test_loss = []
      correct = 0
      count = 0


      for test_batch in tqdm(test_dataloader, desc="Testing"):
        x, y = test_batch
        x = x.float().cuda()
        y = y.cuda()
        logits = model(x)
        loss = F.cross_entropy(logits, y)
        test_loss.append(loss)
        correct += (logits.argmax(dim=-1) == torch.argmax(y, dim=1)).float().sum()
        count += len(y)

      test_loss = torch.mean(torch.tensor(test_loss))
      test_acc = correct / count

      # Log test metrics
      # wandb.log({"test/loss": test_loss, "test/acc": test_acc})

      print()
      print(f"--- TEST ---")
      print("loss: ", test_loss.item())
      print("accuracy: ", test_acc.item())
  
  model.train()
  return array_loss_train, array_loss_val

In [ ]:
mobilenet_v3_large = models.mobilenet_v3_large(pretrained=True)

# reset final fully connected layer
num_ftrs = list(mobilenet_v3_large.classifier.children())[0].in_features

mobilenet_v3_large.classifier = nn.Sequential(
                        nn.Linear(num_ftrs, 256),  
                        nn.ReLU(), 
                        nn.Dropout(hyp["dropout_rate"]),
                        nn.Linear(256, 10))

resize = TransformationLayer(image_transformation = [T.Resize((224,244))])

mobilenet_v3_large = nn.Sequential(
    dataAugumentation,
    resize,
    mobilenet_v3_large
)

mobilenet_v3_large = mobilenet_v3_large.cuda()

In [ ]:
summary(mobilenet_v3_large, (3, 32, 32))

In [ ]:
init_weights_mobilenet = copy.deepcopy(mobilenet_v3_large.state_dict())

In [ ]:
tr_loss, val_los = train(mobilenet_v3_large, train_dataloader, val_dataloader,num_epochs=15, lr = hyp["lerning_rate"])

In [ ]:
figure(figsize=(15,11), dpi=80)
plt.plot(range(1,16), tr_loss,label = "Train loss")
plt.plot(range(1,16), val_los, label = "Validation loss")
plt.title(hyp)
plt.legend()

In [ ]:
epochs_num = np.argmin(val_los)
print("Epoch hyperparameter " + str(epochs_num))

In [ ]:
print("Best val loss")
val_los[epochs_num]

In [ ]:
def prediction(model,test_dataloader):
  with torch.no_grad():
    model.eval()
    prediction = np.empty([0], dtype=np.int8)


    for test_batch in tqdm(test_dataloader, desc="Testing"):
      x, _ = test_batch
      x = x.float().cuda()
      logits = model(x)
      tmp_arr = logits.argmax(dim=-1)
      prediction = np.append(prediction,tmp_arr.cpu().numpy())
  return prediction

In [ ]:
cifar10_dataloader_noShuffle = DataLoader(val_datasets, batch_size=hyp["batch_size"], shuffle=False)

In [ ]:
p = prediction(mobilenet_v3_large, cifar10_dataloader_noShuffle)

In [ ]:
p_encoded = cifar10_dataset.encodeLabel(p)
p2 = np.array([int(np.argmax(i[1])) for i in val_datasets])
p2_encoded = cifar10_dataset.encodeLabel(p2)
cm = confusion_matrix(p_encoded, p2_encoded)
cm

In [ ]:
import seaborn as sns

figure(figsize=(15,11), dpi=80)

ax = sns.heatmap(cm, annot=True, cmap='Blues')

ax.set_title('mobilenet_v3_larges\n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(cifar10_dataset.encodeLabel(list(range(10))))
ax.yaxis.set_ticklabels(cifar10_dataset.encodeLabel(list(range(10))))

## Display the visualization of the Confusion Matrix.
plt.show()

In [ ]:
from itertools import chain
from sklearn.utils import _safe_indexing

indexes = np.arange(len(cifar10_dataset))
cifar10_Whole = list(
        chain.from_iterable(
            (_safe_indexing(a, indexes)) for a in [cifar10_dataset]
        )
      )

In [ ]:
cifar10_dataloader_Whole = DataLoader(cifar10_Whole, batch_size=hyp["batch_size"], shuffle=hyp["shuffle"])

In [ ]:
mobilenet_v3_large.load_state_dict(init_weights_mobilenet)

In [ ]:
tr_loss, _ = train(mobilenet_v3_large, cifar10_dataloader_Whole,num_epochs=epochs_num,lr = hyp["lerning_rate"])

In [ ]:
cifar10_dataset_test = CustomImageDataset("./test/")

In [ ]:
test_dataloader = DataLoader(cifar10_dataset_test, batch_size=hyp["batch_size"], shuffle=False)


In [ ]:
 p_test = prediction(mobilenet_v3_large, test_dataloader)

In [ ]:
def preview(datasets,encoder, labels = None,index = None):
  if index is not None:
    figure(figsize=(8, 6), dpi=80)
    img, label = datasets[index]
    if labels is not None:
      label = labels[index]
    plt.imshow(img.int().T)
    plt.show()
    print(encoder([label]))
    print()
    return

  visited = []

  for i, data in enumerate(datasets): 
    img, label = data
    if labels is not None:
      label = labels[i]
    if label in visited:
      continue
    figure(figsize=(8, 6), dpi=80)
    plt.imshow(img.int().T)
    plt.show()
    print(encoder([label]))
    print()
    visited.append(label)
    if len(visited) >= 10:
      break
    
    

In [ ]:
preview(cifar10_dataset_test,lambda x: cifar10_dataset.encodeLabel(x),p_test)

In [ ]:
df = pd.DataFrame(data=cifar10_dataset.encodeLabel(p_test))
df.columns = ["label"]
df.index += 1 

In [ ]:
df.to_csv('submission.csv', index=True, index_label="id")

In [ ]:
#!kaggle competitions submit -c cifar-10 -f ../kaggle/working/submission.csv -m "Deep Learning, PW, First try"

